In [ ]:
# !pip install segmentation_models_pytorch

In [ ]:
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
from mydataset2 import MyDataset
from tqdm import tqdm
from torch.cuda.amp import autocast
import segmentation_models_pytorch as smp
from data_augmentation import RandomRotation, RandomVerticalFlip, RandomHorizontalFlip, Compose, ToTensor

In [ ]:
batch_size = 1

In [ ]:
train_set = MyDataset(root="data", is_train=True, transform=Compose([
    ToTensor(),
    RandomHorizontalFlip(),
    RandomVerticalFlip(),
    RandomRotation([0, 90, 180, 270]),
]), normalize=transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)))

val_set = MyDataset(root="data", is_train=False, transform=Compose([ToTensor()]),
                    normalize=transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)))

train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0
)

val_loader = DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0
)
len(train_set), len(val_set), len(train_loader), len(val_loader)

In [ ]:
model0 = smp.FPN(
    in_channels=6,
    classes=2,
)
model0.cuda()
criterion0 = nn.CrossEntropyLoss()
optim0 = torch.optim.AdamW(model0.parameters(), lr=0.0001, weight_decay=1e-9)
scaler0 = torch.cuda.amp.GradScaler(enabled=True)

In [ ]:
model1 = smp.DeepLabV3Plus(
    in_channels=6,
    classes=2,
)
model1.cuda()
criterion1 = nn.CrossEntropyLoss()
optim1 = torch.optim.AdamW(model1.parameters(), lr=0.0001, weight_decay=1e-9)
scaler1 = torch.cuda.amp.GradScaler(enabled=True)

In [ ]:
model2 = smp.UnetPlusPlus(
    in_channels=6,
    classes=2,
)
model2.cuda()
criterion2 = nn.CrossEntropyLoss()
optim2 = torch.optim.AdamW(model2.parameters(), lr=0.0001, weight_decay=1e-9)
scaler2 = torch.cuda.amp.GradScaler(enabled=True)

In [ ]:
class MetaModel(nn.Module):
    def __init__(self):
        super(MetaModel, self).__init__()
        self.l1 = nn.Linear(6, 16)
        self.l2 = nn.Linear(16, 2)

    def forward(self, x):
        x = x.view((-1, 6, 256, 256))
        x = x.permute((0, 2, 3, 1))
        x = torch.relu(self.l1(x))
        x = self.l2(x)
        x = x.permute((0, 3, 1, 2))
        return x


In [ ]:
class Stacking(nn.Module):
    def __init__(self, base_models, meta_model):
        super(Stacking, self).__init__()
        self.base_models = nn.ModuleList(base_models)
        self.meta_model = meta_model

    def forward(self, x):
        outputs = torch.zeros(100, 2, 256, 256)
        for model in self.base_models:
            output = model(x)
            outputs = torch.add(outputs, output)
        # x = self.meta_model(x)
        return outputs

In [ ]:
# models = [model0, model1, model2]
# criterions = [criterion0, criterion1, criterion2]
# scalers = [scaler0, scaler1, scaler2]
# optims = [optim0, optim1, optim2]
#
# metamodel = MetaModel()
# metamodel.cuda()
# meta_criterion = nn.CrossEntropyLoss()
# meta_optim = torch.optim.AdamW(metamodel.parameters(), lr=0.001, weight_decay=1e-9)
# meta_scaler = torch.cuda.amp.GradScaler(enabled=True)

In [ ]:
metamodel = MetaModel()
metamodel.cuda()
model = Stacking(
    [model0, model1, model2], metamodel
)
criterion = nn.CrossEntropyLoss()
optim = torch.optim.AdamW(metamodel.parameters(), lr=0.001, weight_decay=1e-9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim, mode='max', factor=0.1, patience=2,
    verbose=True)
scaler = torch.cuda.amp.GradScaler(enabled=True)

In [ ]:
def train_model(epoch):
    model.train()
    print(f"Epoch {epoch} Training")
    with tqdm(train_loader, desc=str(epoch)) as it:
        for img, mask in it:
            img, mask = img.cuda(), mask.cuda()
            optim.zero_grad()
            mask = mask.long()
            with autocast():
                outputs = model(img)
                mask = mask.squeeze(1)
                loss = criterion(outputs, mask)
            # loss.backward()
            # optim.step()
            scaler.scale(loss).backward()
            scaler.step(optim)
            scaler.update()
            #print(outputs.data.shape)
            _, pred = torch.max(outputs.data, 1)
            #print(pred.shape)
            p, r, f1, iou = get_index(pred, mask)
            it.set_postfix_str(f"loss: {loss.item(): .4f} p: {p: .4f}  r: {r: .4f}  f1: {f1: .4f}  iou: {iou: .4f}")

In [ ]:
def get_index(pred, label):
    eps = 1e-7
    tp = torch.sum(label * pred)
    fp = torch.sum(pred) - tp
    fn = torch.sum(label) - tp

    p = (tp + eps) / (tp + fp + eps)
    r = (tp + eps) / (tp + fn + eps)
    f1 = (2 * p * r + eps) / (p + r + eps)
    iou = (tp + eps) / (tp + fn + fp + eps)
    return p, r, f1, iou


def test_model(epoch):
    model.eval()
    global max_score
    f1s = 0
    print(f"Epoch {epoch} Testing")
    with torch.no_grad():
        with tqdm(val_loader, desc=str(epoch)) as it:
            for img, mask in it:
                img, mask = img.cuda(), mask.cuda()
                outputs = model(img)
                _, pred = torch.max(outputs.data, 1)
                mask = mask.squeeze(1)
                p, r, f1, iou = get_index(pred, mask)
                f1s += f1
                it.set_postfix_str(f"p: {p: .4f}  r: {r: .4f}  f1: {f1: .4f}  iou: {iou: .4f}")
    f1s /= len(val_loader)
    scheduler.step(f1s)
    print("f1", f1s.item())
    if max_score < f1s:
        max_score = f1s
        print('max_score', max_score.item())

In [ ]:
num_epoch = 10
max_score = 0
for epoch in range(0, num_epoch):
    train_model(epoch=epoch)
    test_model(epoch=epoch)
print("completed!")
print('max_score', max_score.item())